# Tablesaw 

[Tablesaw](https://tablesaw.tech/) is easy to add to the BeakerX Groovy kernel.
Tablesaw provides the ability to easily transform, summarize, and filter data, as well as computing descriptive statistics and fundamental machine learning algorithms.

This notebook has some basic demos of how to use Tablesaw, including visualizing the results.  This notebook uses the Beaker interactive visualizaiton libraries, but Tablesaw's APIs also work.  The notebook covers basic table manipulation, k-means clustering, and linear regression.

In [3]:
%classpath add mvn tech.tablesaw tablesaw-plot 0.10.0
%classpath add mvn tech.tablesaw tablesaw-smile 0.10.0
%import tech.tablesaw.api.*
%import tech.tablesaw.api.ml.clustering.*
%import tech.tablesaw.api.ml.regression.*
%import tech.tablesaw.columns.*    
%import tech.tablesaw.reducing.*    

Added jars: [smile-plot.jar, xchart.jar, filters.jar, swingx.jar, swing-worker.jar, tablesaw-smile.jar, VectorGraphics2D.jar, tablesaw-plot.jar]


In [ ]:
tornadoes = Table.read().csv("Tablesaw/tornadoes_2014.csv")

In [ ]:
//print dataset structure
tornadoes.structure()

In [ ]:
//get header names
tornadoes.columnNames()

In [ ]:
//displays the row and column counts
tornadoes.shape()

In [ ]:
//displays the first n rows
tornadoes.first(10)

In [ ]:
import static tech.tablesaw.api.QueryHelper.column
tornadoes.structure().selectWhere(column("Column Type").isEqualTo("FLOAT"))

In [ ]:
//summarize the data in each column
tornadoes.summary()

In [ ]:
//Mapping operations
def month = tornadoes.dateColumn("Date").month()
tornadoes.addColumn(month);
tornadoes.columnNames()

In [ ]:
//Sorting by column
tornadoes.sortOn("-Fatalities")

In [ ]:
//Descriptive statistics
tornadoes.column("Fatalities").summary()

In [ ]:
//Performing totals and sub-totals
def injuriesByScale = tornadoes.median("Injuries").by("Scale")
injuriesByScale.setName("Median injuries by Tornado Scale")
injuriesByScale

In [ ]:
//Cross Tabs
CrossTab.xCount(tornadoes, tornadoes.categoryColumn("State"), tornadoes.shortColumn("Scale"))

## K-means clustering

K-means is the most common form of “centroid” clustering. Unlike classification, clustering is an unsupervised learning method. The categories are not predetermined. Instead, the goal is to search for natural groupings in the dataset, such that the members of each group are similar to each other and different from the members of the other groups. The K represents the number of groups to find.

We’ll use a well known Scotch Whiskey dataset, which is used to cluster whiskeys according to their taste based on data collected from tasting notes. As always, we start by loading data and printing its structure.

More description is available at https://jtablesaw.wordpress.com/2016/08/08/k-means-clustering-in-java/

In [ ]:
t = Table.read().csv("Tablesaw/whiskey.csv")
t.structure()

In [ ]:
model = new Kmeans(
    5,
    t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
    t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
);

//print claster formation
model.clustered(t.column("Distillery"));

In [ ]:
//print centroids for each claster
model.labeledCentroids();

In [ ]:
//gets the distortion for our model
model.distortion()

In [ ]:
def n = t.rowCount();
def kValues = new double[n - 2];
def distortions = new double[n - 2];

for (int k = 2; k < n; k++) {
  kValues[k - 2] = k;
  def kmeans = new Kmeans(k,
      t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
      t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
  );
  distortions[k - 2] = kmeans.distortion();
}
def linearYPlot = new Plot(title: "K-means clustering demo", xLabel:"K", yLabel: "distortion")
linearYPlot << new Line(x: kValues, y: distortions)

## Play (Money)ball with Linear Regression

In baseball, you make the playoffs by winning more games than your rivals. The number of games the rivals win is out of your control so the A’s looked instead at how many wins it took historically to make the playoffs. They decided that 95 wins would give them a strong chance.  Here’s how we might check that assumption in Tablesaw.

More description is available at https://jtablesaw.wordpress.com/2016/07/31/play-moneyball-data-science-in-tablesaw/

In [ ]:
import static tech.tablesaw.api.QueryHelper.column

baseball = Table.read().csv("Tablesaw/baseball.csv");

// filter to the data available at the start of the 2002 season
moneyball = baseball.selectWhere(column("year").isLessThan(2002));
wins = moneyball.nCol("W");
year = moneyball.nCol("Year");
playoffs = moneyball.column("Playoffs");
runDifference = moneyball.shortColumn("RS").subtract(moneyball.shortColumn("RA"));
moneyball.addColumn(runDifference);
runDifference.setName("RD");

def Plot = new Plot(title: "RD x Wins", xLabel:"RD", yLabel: "W")
Plot << new Points(x: moneyball.numericColumn("RD").toDoubleArray(), y: moneyball.numericColumn("W").toDoubleArray())

In [ ]:
winsModel = LeastSquares.train(wins, runDifference);

In [ ]:
def runDiff = new double[1];
runDiff[0] = 135;
def expectedWins = winsModel.predict(runDiff);
runsScored2 = LeastSquares.train(moneyball.nCol("RS"), moneyball.nCol("OBP"), moneyball.nCol("SLG"));

In [ ]:
new Histogram(xLabel:"X",
              yLabel:"Proportion",
              data: Arrays.asList(runsScored2.residuals()), 
              binCount: 25);

## Financial and Economic Data

You can fetch data from [Quandl](https://www.quandl.com/) and load it directly into Tablesaw

In [1]:
%classpath add mvn com.jimmoores quandl-tablesaw 2.0.0
%import com.jimmoores.quandl.*
%import com.jimmoores.quandl.tablesaw.*

Added jars: [RoaringBitmap.jar, javax.ws.rs-api.jar, threetenbp.jar, annotations.jar, jersey-guava.jar, tablesaw-core.jar, j2objc-annotations.jar, error_prone_annotations.jar, animal-sniffer-annotations.jar, commons-collections.jar, javassist.jar, kotlin-stdlib-jre7.jar, kotlin-stdlib.jar, javax.inject.jar, quandl-tablesaw.jar, smile-math.jar, jackson-annotations.jar, jsr305.jar, fastutil.jar, smile-graph.jar, jackson-databind.jar, kotlin-stdlib-jre8.jar, commons-lang3.jar, jersey-client.jar, commons-logging.jar, jackson-datatype-json-org.jar, smile-core.jar, commons-math3.jar, commons-text.jar, jersey-common.jar, guava.jar, opencsv.jar, gson.jar, slf4j-api.jar, quandl-core.jar, hk2-api.jar, osgi-resource-locator.jar, jackson-core.jar, json.jar, smile-data.jar, hk2-locator.jar, hk2-utils.jar, aopalliance-repackaged.jar, snappy.jar, javax.annotation-api.jar, commons-beanutils.jar]


In [4]:
TableSawQuandlSession session = TableSawQuandlSession.create();
Table table = session.getDataSet(DataSetRequest.Builder.of("WIKI/AAPL").build());
// Create a new column containing the year
ShortColumn yearColumn = table.dateColumn("Date").year();
yearColumn.setName("Year");
table.addColumn(yearColumn);
// Create max, min and total volume tables aggregated by year
Table summaryMax = table.max("Close").by("year");
Table summaryMin = table.minimum("Close").by("year");
Table summaryVolume = table.sum("Volume").by("year");
// Create a new table from each of these
summary = Table.create("Summary", summaryMax.column(0), summaryMax.column(1), 
                       summaryMin.column(1), summaryVolume.column(1));
// Add back a DateColumn to the summary...will be used for plotting
DateColumn yearDates = new DateColumn("YearDate");
for(year in summary.column('Year')){
    yearDates.append(java.time.LocalDate.of(year,1,1));
}
summary.addColumn(yearDates)

summary


 Year  |  Max [Close]  |  Min [Close]  |  Sum [Volume]   |   YearDate   |
-------------------------------------------------------------------------
 1980  |         36.0  |        25.25  |      6003800.0  |  1980-01-01  |
 1981  |         34.5  |        14.25  |      3.65935E7  |  1981-01-01  |
 1982  |        33.87  |         11.0  |    9.5379504E7  |  1982-01-01  |
 1983  |        62.75  |        17.88  |     1.860744E8  |  1983-01-01  |
 1984  |        33.25  |        21.87  |     1.874064E8  |  1984-01-01  |
 1985  |        30.62  |         14.5  |   2.03094976E8  |  1985-01-01  |
 1986  |        43.75  |        22.12  |   2.38050096E8  |  1986-01-01  |
 1987  |        80.25  |         28.0  |   3.84217088E8  |  1987-01-01  |
 1988  |        47.25  |        36.13  |   3.68687296E8  |  1988-01-01  |
 1989  |        49.63  |        33.75  |   4.54516288E8  |  1989-01-01  |
 1990  |        47.38  |         25.0  |   3.96445888E8  |  1990-01-01  |
 1991  |        72.75  |        41.13

In [5]:
years = summary.column('YearDate').collect{ Date.from(it.atStartOfDay().atZone(java.time.ZoneId.systemDefault()).toInstant()) }

plot = new TimePlot(title: 'Price Chart for AAPL', xLabel: 'Time', yLabel: 'Max [Close]')
plot.add(new YAxis(label: 'Volume'))
plot.add(new Points(x: years, y: summary.column('Max [Close]').collect()))
plot.add(new Line(x: years, y: summary.column('Max [Close]').collect(), color: Color.blue))
plot.add(new Bars(x: years, y: summary.column('Sum [Volume]').collect(), yAxis: 'Volume'))


